In [3]:
!pip install pandas numpy matplotlib seaborn xverse woe

# ### Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [6]:
import pandas as pd

# Directly read the file
df = pd.read_csv('data.csv')
print("Data loaded successfully!")
print(f"Shape: {df.shape}")
df.head()

Data loaded successfully!
Shape: (95662, 16)


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [16]:
df.head()
df.info()
df.describe(include='all')

# Check missing values
missing = df.isnull().sum()
print(f"Missing Values:\n{missing[missing > 0]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   TransactionId         95662 non-null  object             
 1   BatchId               95662 non-null  object             
 2   AccountId             95662 non-null  object             
 3   SubscriptionId        95662 non-null  object             
 4   CustomerId            95662 non-null  object             
 5   CurrencyCode          95662 non-null  object             
 6   CountryCode           95662 non-null  int64              
 7   ProviderId            95662 non-null  object             
 8   ProductId             95662 non-null  object             
 9   ProductCategory       95662 non-null  object             
 10  ChannelId             95662 non-null  object             
 11  Amount                95662 non-null  float64            
 12  Valu

In [17]:
# Identifying Missing Values
missing_values = df.isnull().sum()
print("\nMissing Values:")
print(missing_values[missing_values > 0])


Missing Values:
Series([], dtype: int64)


Feature Engineering

In [9]:
# ### Create Aggregate Features
customer_df = df.groupby('CustomerId').agg(
    total_transactions=('TransactionId', 'count'),
    avg_amount=('Amount', 'mean'),
    max_amount=('Amount', 'max'),
    product_categories=('ProductCategory', 'nunique'),
    fraud_count=('FraudResult', 'sum')
).reset_index()


In [14]:
# ### Temporal Features
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])
df['transaction_hour'] = df['TransactionStartTime'].dt.hour
df['transaction_day'] = df['TransactionStartTime'].dt.day
df['transaction_month'] = df['TransactionStartTime'].dt.month

In [21]:

numeric_df = customer_df.drop('CustomerId', axis=1)

imputer = SimpleImputer(strategy='median')
imputed_data = imputer.fit_transform(numeric_df)

imputed_df = pd.DataFrame(imputed_data, columns=numeric_df.columns)

# Merge back with the original DataFrame
customer_df = pd.concat([customer_df[['CustomerId']], imputed_df], axis=1)

In [23]:
# --- Feature Engineering ---

# Create aggregate features (EXCLUDE CustomerId from aggregation)
customer_features = df.groupby('CustomerId').agg(
    total_transactions=('TransactionId', 'count'),
    avg_amount=('Amount', 'mean'),
    max_amount=('Amount', 'max'),
    product_categories=('ProductCategory', 'nunique'),
    fraud_count=('FraudResult', 'sum')
).reset_index()

# --- Handle Missing Values ---

# Separate CustomerId from features
customer_ids = customer_features['CustomerId']
features = customer_features.drop('CustomerId', axis=1)

# Impute only numerical features
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
features_imputed = imputer.fit_transform(features)

# Rebuild DataFrame
customer_df = pd.DataFrame(features_imputed, columns=features.columns)
customer_df = pd.concat([customer_ids, customer_df], axis=1)

# Verify
print("Missing values after imputation:", customer_df.isnull().sum().sum())

Missing values after imputation: 0


In [24]:
# Check data types
print(customer_df.dtypes)

# If any non-numeric features remain, convert them:
# customer_df['column_name'] = customer_df['column_name'].astype(float)

CustomerId             object
total_transactions    float64
avg_amount            float64
max_amount            float64
product_categories    float64
fraud_count           float64
dtype: object


In [25]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Assuming 'customer_df' is your DataFrame

# --- Handle Missing Values ---

# Separate CustomerId from features
customer_ids = customer_df['CustomerId']
features = customer_df.drop('CustomerId', axis=1)

# Impute only numerical features
imputer = SimpleImputer(strategy='median')
features_imputed = imputer.fit_transform(features)

# Rebuild DataFrame
customer_df_imputed = pd.DataFrame(features_imputed, columns=features.columns)
customer_df_imputed = pd.concat([customer_ids, customer_df_imputed], axis=1)

# Verify
print("Missing values after imputation:", customer_df_imputed.isnull().sum().sum())

# --- Save Data ---

# Save to CSV
customer_df_imputed.to_csv('customer_data_imputed.csv', index=False)
print("Data saved to customer_data_imputed.csv")

Missing values after imputation: 0
Data saved to customer_data_imputed.csv
